In [35]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()

spark= SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [36]:
loan_defaulters_df = spark.read \
.format("csv") \
.option("inferSchema",True) \
.option("header",True) \
.load("/public/trendytech/lendingclubproject/raw/loans_defaulters_csv")


In [37]:
loan_defaulters_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- delinq_amnt: double (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- mths_since_last_delinq: string (nullable = true)
 |-- mths_since_last_record: string (nullable = true)



In [38]:
loan_defaulters_df.createOrReplaceTempView("loan_defaulter")

In [39]:
loan_defaulters_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- delinq_amnt: double (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- mths_since_last_delinq: string (nullable = true)
 |-- mths_since_last_record: string (nullable = true)



In [40]:
spark.sql("select distinct(delinq_2yrs) from loan_defaulter")

delinq_2yrs
1.0
I bike to work on...
271 monthly payme...
183xx
VISA and AMEX cre...
etc. and I feel t...
AZ
017xx
923xx
446xx


In [41]:
spark.sql("select delinq_2yrs,count(*) as total from loan_defaulter group by delinq_2yrs order by total desc ")

delinq_2yrs,total
0.0,1838878
1.0,281335
2.0,81285
3.0,29539
4.0,13179
5.0,6599
6.0,3717
7.0,2062
8.0,1223
9.0,818


In [42]:
loan_defaulters_schema = "member_id string, delinq_2yrs float, delinq_amnt float, pub_rec float, pub_rec_bankruptcies float,inq_last_6mths float, total_rec_late_fee float, mths_since_last_delinq float, mths_since_last_record float"

In [43]:
loans_def_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.schema(loan_defaulters_schema) \
.load("/public/trendytech/lendingclubproject/raw/loans_defaulters_csv")

In [44]:
from pyspark.sql.functions import current_timestamp

loans_default_ingested_df = loans_def_raw_df.withColumn("ingest_date", current_timestamp())

In [45]:
loans_default_ingested_df.createOrReplaceTempView("loan_defaulter")

In [46]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulter group by delinq_2yrs order by total desc").show(40)

+-----------+-------+
|delinq_2yrs|  total|
+-----------+-------+
|        0.0|1838878|
|        1.0| 281335|
|        2.0|  81285|
|        3.0|  29539|
|        4.0|  13179|
|        5.0|   6599|
|        6.0|   3717|
|        7.0|   2062|
|        8.0|   1223|
|        9.0|    818|
|       10.0|    556|
|       11.0|    363|
|       12.0|    264|
|       null|    261|
|       13.0|    165|
|       14.0|    120|
|       15.0|     87|
|       16.0|     55|
|       17.0|     30|
|       18.0|     30|
|       19.0|     23|
|       20.0|     17|
|       21.0|     12|
|       22.0|      5|
|       24.0|      4|
|       26.0|      3|
|       29.0|      2|
|       25.0|      2|
|       23.0|      2|
|       3.44|      2|
|       30.0|      2|
|       39.0|      1|
|      17.18|      1|
|      18.53|      1|
|      21.72|      1|
|       5.52|      1|
|       1.37|      1|
|      13.76|      1|
|      22.95|      1|
|       3.45|      1|
+-----------+-------+
only showing top 40 rows



In [47]:
from pyspark.sql.functions import *

In [48]:
spark.sql("select count(*) from loan_defaulter where delinq_2yrs is null")

count(1)
261


In [54]:
loans_def_processed_df = loans_default_ingested_df.withColumn("delinq_2yrs", col("delinq_2yrs").cast("integer")).fillna(0, subset = ["delinq_2yrs"])

In [55]:
loans_def_processed_df.createOrReplaceTempView("loan_defaulter")

In [56]:
spark.sql("select count(*) from loan_defaulter where delinq_2yrs is null")

count(1)
0


In [57]:
spark.sql("select * from loan_defaulter")

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record,ingest_date
9cb79aa7323e81be1...,2,0.0,0.0,0.0,0.0,0.0,11.0,null,2025-03-30 08:35:...
0dd2bbc517e3c8f9e...,0,0.0,1.0,1.0,3.0,0.0,null,115.0,2025-03-30 08:35:...
458458599d3df3bfc...,0,0.0,1.0,1.0,1.0,0.0,null,76.0,2025-03-30 08:35:...
05ea141ec28b5c7f7...,0,0.0,0.0,0.0,0.0,0.0,null,null,2025-03-30 08:35:...
aac68850fdac09fd0...,1,0.0,0.0,0.0,0.0,0.0,21.0,null,2025-03-30 08:35:...
3a423e4589e89f429...,0,0.0,0.0,0.0,0.0,0.0,null,null,2025-03-30 08:35:...
f1efcf7dfbfef21be...,0,0.0,0.0,0.0,1.0,0.0,null,null,2025-03-30 08:35:...
c89986155a070db2e...,1,0.0,0.0,0.0,1.0,15.0,5.0,null,2025-03-30 08:35:...
118dc629b6e134419...,0,0.0,0.0,0.0,0.0,0.0,null,null,2025-03-30 08:35:...
a86fa4b7493708333...,0,0.0,0.0,0.0,0.0,0.0,null,null,2025-03-30 08:35:...


In [58]:
loans_def_delinq_df = spark.sql("select member_id, delinq_2yrs, delinq_amnt, int(mths_since_last_delinq), ingest_date from loan_defaulter where delinq_2yrs >0 and mths_since_last_delinq > 0")

In [62]:
loans_def_records_enq_df = spark.sql("select member_id from loan_defaulter where pub_rec > 0.0 or pub_rec_bankruptcies > 0.0 or inq_last_6mths > 0.0")

In [63]:
loans_def_records_enq_df.count()

1070125

In [65]:
loans_def_records_enq_df.write \
.option("header", True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv015703/lendingclubproject/raw/cleaned/loans_defaulters_deling_csv") \
.save()

In [66]:
loans_def_records_enq_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv015703/lendingclubproject/raw/cleaned/loans_defaulters_deling_parquet") \
.save()


In [67]:
loans_def_records_enq_df.write \
.option("header", True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv015703/lendingclubproject/raw/cleaned/loans_defaulters_records_enq_csv") \
.save()

In [68]:
loans_def_records_enq_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv015703/lendingclubproject/raw/cleaned/loans_defaulters_records_enq_parquet") \
.save()

In [70]:
loans_def_detail_record_delinq_df = spark.sql("select member_id, pub_rec, pub_rec_bankruptcies, inq_last_6mths, ingest_date from loan_defaulter where pub_rec > 0.0 or pub_rec_bankruptcies > 0.0 or inq_last_6mths > 0.0")

In [71]:
loans_def_processed_public_record_df = loans_def_detail_record_delinq_df.withColumn("pub_rec", col("pub_rec").cast("integer")).fillna(0, subset = ["pub_rec"])
loans_def_processed_public_record_bankruptcies_df = loans_def_processed_public_record_df.withColumn("pub_rec_bankruptcies", col("pub_rec_bankruptcies").cast("integer")).fillna(0, subset = ["pub_rec_bankruptcies"])
loans_def_processed_public_record_inq6mths_df = loans_def_processed_public_record_bankruptcies_df.withColumn("inq_last_6mths", col("inq_last_6mths").cast("integer")).fillna(0, subset = ["inq_last_6mths"])
loans_def_processed_public_record_inq6mths_df.createOrReplaceTempView("loan_defaulters")
loans_def_detail_record_final_df = spark.sql("select member_id, pub_rec, pub_rec_bankruptcies, inq_last_6mths, ingest_date from loan_defaulters")

In [73]:
loans_def_detail_record_final_df.write\
.format("parquet")\
.mode("overwrite")\
.option("path", "/user/itv015703/lendingclubproject/cleaned/loan_defaulters_detail_records_enq_parquet")\
.save()

In [74]:
loans_def_detail_record_final_df.write\
.option("header", True)\
.format("csv")\
.mode("overwrite")\
.option("path", "/user/itv015703/lendingclubproject/cleaned/loan_defaulters_detail_records_enq_csv")\
.save()

In [75]:
spark.stop()